Setup MODIN

In [1]:
import os

os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

import pandas as pd

Import Libraries

In [2]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 80
pd.options.display.max_rows = 80
import seaborn as sns
%matplotlib inline
sns.set()
def dfy(s):
    return pd.DataFrame(s)

In /opt/myintel/intelpython/latest/envs/nntf/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/myintel/intelpython/latest/envs/nntf/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/myintel/intelpython/latest/envs/nntf/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/myintel/intelpython/latest/envs/nntf/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor 

In [3]:
data = pd.read_pickle('cooked.pkl')

In [4]:
category_columns = [x for x in data.columns if 'F_2_' in x]

In [5]:
data.loc[:,category_columns] = data.loc[:,category_columns].astype(np.uint8)

In [6]:
dm_1 = data[data.missing_cols==1]
dc = data[data.missing_cols==0]

#### Make datasets for single column learning<br>
 - Methods possible other than imputation:
 - PCA(dimensionality reduction)
 - feature elimination

Initial Steps<br>
 - [F]_clean stores indexes where each column is not missing
 - Separate them based on number of missing columns in each row

In [7]:
missing_ = data.isna().sum(axis=0)
missing_features = np.array(missing_[missing_ > 0].index)

    missing_features = array(['F_1_0', 'F_1_1', 'F_1_2', 'F_1_3', 'F_1_4', 'F_1_5', 'F_1_6',
       'F_1_7', 'F_1_8', 'F_1_9', 'F_1_10', 'F_1_11', 'F_1_12', 'F_1_13',
       'F_1_14', 'F_3_0', 'F_3_1', 'F_3_2', 'F_3_3', 'F_3_4', 'F_3_5',
       'F_3_6', 'F_3_7', 'F_3_8', 'F_3_9', 'F_3_10', 'F_3_11', 'F_3_12',
       'F_3_13', 'F_3_14', 'F_3_15', 'F_3_16', 'F_3_17', 'F_3_18',
       'F_3_19', 'F_3_20', 'F_3_21', 'F_3_22', 'F_3_23', 'F_3_24',
       'F_4_0', 'F_4_1', 'F_4_2', 'F_4_3', 'F_4_4', 'F_4_5', 'F_4_6',
       'F_4_7', 'F_4_8', 'F_4_9', 'F_4_10', 'F_4_11', 'F_4_12', 'F_4_13',
       'F_4_14'], dtype=object)

In [8]:
F_clean = {f:data[data[f].isna()==False].index for f in missing_features}
F_dirty = {f:data[data[f].isna()].index for f in missing_features}

In [9]:
missing_.sort_values(inplace=True)

#### Missing Data Distribution
![Image](missing_data_distribution.png)

#### Densities of the least missing features
##### F_1_10 and F_4_4
![Image](F_1_0density.png)![Image](F_4_4.png)

In [10]:
training_order = missing_.index

#### Next Steps 
 - Developed Training Order
 - need to examine predictability of lower features 

In [11]:
def feature_correlations(f_):
    clean_f110 =dc[f_].copy()
    # dc_corr = dc.corrwith(clean_f110, axis=1)
    # fig = sns.heatmap(dc_corr)
    correls_clean_f110 = {}
    for f in dc.columns:
        if f != f_:
            correls_clean_f110[f] = np.corrcoef(clean_f110,dc[f])[0][1]
    correls_clean_f110.pop('missing_cols')

    plt.figure(figsize=(18,18))
    fig = sns.barplot(y=list(correls_clean_f110.keys()),x=list(correls_clean_f110.values()), orient='h')

#### Feature correlations('F_1_10')
![Image](feature_correlations('F_1_10').png)

#### Feature correlations('F_4_4')
![Image](feature_correlations('F_4_4').png)

In [12]:
# Correlation in different scenarios
corr_data = data.drop(['missing_cols'],axis=1).corr()
corr_clean = dc.drop(['missing_cols'],axis=1).corr()

In [13]:
# percent_diff_in_correlation
pdic = (corr_data.abs() - corr_clean.abs())/corr_clean.abs()

In [16]:
# finding the columns with maximum and minimum change
correlation_variance_cols = {}
for f in pdic.columns:
    pdic_np = pdic[f].to_numpy()
    correlation_variance_cols[f] = {'max_col': pdic.columns[pdic_np.argmax()],
                               'min_col':pdic.columns[pdic_np.argmin()]}
    # finding the magitude
variation_magnitude={}
for f in pdic.columns:
    pdic_np = pdic[f].to_numpy()
    variation_magnitude[f] = {'max_col': pdic_np.max(),
                               'min_col':pdic_np.min()}
    correlation_vector_distance = {}
for f in pdic.columns:
    pdic_np = pdic[f].to_numpy()
    correlation_vector_distance[f] = np.linalg.norm(pdic_np)**0.5

In [ ]:
dfy(correlation_variance_cols)

In [ ]:
dfy(variation_magnitude)

#### Correlations EDA
![Image](max_corr_var.png)
![Image](sdfs.png)


>Correlation in total Data vs Correlation in non-missing data slice varies heavily for certain features
>>Deviation_distance is needed for exploration

#### Deviation Distance
![Image](devd-Copy1.png)

sorted(correlation_vector_distance,key=correlation_vector_distance.get)

In [ ]:
correlation_vector_distance['F_4_3']

In [ ]:
mcov = data.cov()
# for i in range(10):
#     d_state = data[data.missing_cols==i]
#     d_state_corr = d_state.cov()

data_ = data.copy()  
dc_ = dc.copy()

In [ ]:
varis = list(set(data.columns)-set(category_columns)-set(['missing_cols']))

    varis =['F_4_12', 'F_4_10', 'F_3_19', 'F_3_21', 'F_3_9', 'F_1_9', 'F_3_18', 'F_1_0', 'F_4_7', 'F_1_5', 'F_1_11', 'F_3_8', 'F_3_1', 'F_3_17', 'F_4_9', 'F_4_13', 'F_1_2', 'F_4_8', 'F_3_15', 'F_1_10', 'F_4_6', 'F_1_8', 'F_3_7', 'F_3_13', 'F_3_6', 'F_3_23', 'F_1_6', 'F_3_12', 'F_1_12', 'F_3_4', 'F_1_1', 'F_3_20', 'F_1_13', 'F_4_3', 'F_4_5', 'F_3_2', 'F_4_11', 'F_3_14', 'F_4_1', 'F_1_4', 'F_1_3', 'F_4_14', 'F_4_0', 'F_3_5', 'F_3_3', 'F_4_4', 'F_4_2', 'F_3_24', 'F_3_22', 'F_3_16', 'F_3_11', 'F_3_0', 'F_1_14', 'F_1_7', 'F_3_10']

In [17]:
chd = pd.read_pickle('cookeder.pkl')

In [ ]:
description = data.describe()

In [ ]:
description

In [18]:
from joblib import parallel_backend
from sklearn.neighbors import KernelDensity
# with parallel_backend('loky'):
    # kde = KernelDensity(kernel='gaussian', bandwidth=0.96).fit(dc__)

In [ ]:
chd__ = chd.iloc[:50,:]

In [ ]:
dc__ = dc.drop(['missing_cols'], axis=1)
chd__ = chd__.drop(['missing_cols'], axis=1)

In [ ]:
kde.score_samples(chd__.loc[[1,2],:])

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error,make_scorer
from sklearnex.model_selection import train_test_split
import sklearn
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV,RandomizedSearchCV,HalvingGridSearchCV,ParameterSampler
sklearn.set_config(display='diagram')

X_y = data.loc[F_clean['F_1_10'],:]
X = X_y.drop(['F_1_10'],axis=1)
y = X_y['F_1_10']
rmse= make_scorer(mean_squared_error, squared=False)

histr = HistGradientBoostingRegressor(loss='squared_error',
    max_bins=255,
    categorical_features=cat_idx,
    early_stopping='auto',
    scoring=rmse,
    n_iter_no_change=50,
    random_state=42,)



params = {'learning_rate': [1.0, 0.1, 0.010000000000000002, 0.0010000000000000002], 
          'max_iter': [20, 30, 40], 
          'max_leaf_nodes': [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
          'min_samples_leaf': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 
          'l2_regularization': np.array([0.        , 0.13793103, 0.27586207, 0.4137931 , 0.55172414,
       0.68965517, 0.82758621, 0.96551724, 1.10344828, 1.24137931,
       1.37931034, 1.51724138, 1.65517241, 1.79310345, 1.93103448,
       2.06896552, 2.20689655, 2.34482759, 2.48275862, 2.62068966,
       2.75862069, 2.89655172, 3.03448276, 3.17241379, 3.31034483,
       3.44827586, 3.5862069 , 3.72413793, 3.86206897, 4.        ]),
         'validation_fraction': [0.1, 0.2, 0.3, 0.4]}


with parallel_backend('loky'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    fets = list(X_train.columns)
    cat_idx = []
    for c,m in enumerate(fets):
        if m in category_columns:
            cat_idx.append(c)
    hgbrt = HistGradientBoostingRegressor(loss='squared_error',
                max_bins=255,
                categorical_features=cat_idx,
                early_stopping='auto',
                scoring=rmse,
                n_iter_no_change=50,
                random_state=42,)
    histr = HalvingRandomSearchCV(hgbrt,param_distributions=params,random_state=0,scoring=rmse,n_jobs=-1,return_train_score=True)
    histr.fit(X_train,y_train)
    rmse = mean_squared_error(histr.predict(X_test),y_test,squared=False)
    X_y__ = data.loc[F_dirty['F_1_10'], :]
    X__ = X_y__.drop(['F_1_10'],axis=1)
    y__ = X_y__['F_1_10']
    y2 = chd.loc[X__.index,'F_1_10']
    y_pred = histr.predict(X__)

In [67]:
from sklearn.metrics import make_scorer


In [73]:
params = dict(learning_rate=[0.1**i for i in range(4)],
    max_iter=[10*i for i in range(2,5)],
    max_leaf_nodes=[i for i in range(31,50)],
    min_samples_leaf=[i for i in range(5,40)],
    l2_regularization=np.linspace(0,4,num=30),
    max_bins=255,
    early_stopping='auto',
    validation_fraction=[0.1,0.2,0.3,0.4],)
print(params)

{'learning_rate': [1.0, 0.1, 0.010000000000000002, 0.0010000000000000002], 'max_iter': [20, 30, 40], 'max_leaf_nodes': [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'min_samples_leaf': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'l2_regularization': array([0.        , 0.13793103, 0.27586207, 0.4137931 , 0.55172414,
       0.68965517, 0.82758621, 0.96551724, 1.10344828, 1.24137931,
       1.37931034, 1.51724138, 1.65517241, 1.79310345, 1.93103448,
       2.06896552, 2.20689655, 2.34482759, 2.48275862, 2.62068966,
       2.75862069, 2.89655172, 3.03448276, 3.17241379, 3.31034483,
       3.44827586, 3.5862069 , 3.72413793, 3.86206897, 4.        ]), 'max_bins': 255, 'early_stopping': 'auto', 'validation_fraction': [0.1, 0.2, 0.3, 0.4]}


In [58]:
c_y = chd.loc[X_test.index,'F_1_10']
import gc
gc.collect()

1437

In [62]:
X_y__ = data.loc[F_dirty['F_1_10'], :]
X__ = X_y__.drop(['F_1_10'],axis=1)
y__ = X_y__['F_1_10']
y2 = chd.loc[X__.index,'F_1_10']
y_pred = histr.predict(X__)

In [63]:
mean_squared_error(y2,y_pred,squared=False)

0.001906206689521487

In [68]:
HalvingRandomSearchCV(hgbrt,param_distributions=params,random_state=0,scoring=rmse,n_jobs=-1,return_train_score=True)

Init signature:
HalvingRandomSearchCV(
    estimator,
    param_distributions,
    *,
    n_candidates='exhaust',
    factor=3,
    resource='n_samples',
    max_resources='auto',
    min_resources='smallest',
    aggressive_elimination=False,
    cv=5,
    scoring=None,
    refit=True,
    error_score=nan,
    return_train_score=True,
    random_state=None,
    n_jobs=None,
    verbose=0,
)
Docstring:     
Randomized search on hyper parameters.

The search strategy starts evaluating all the candidates with a small
amount of resources and iteratively selects the best candidates, using more
and more resources.

The candidates are sampled at random from the parameter space and the
number of sampled candidates is determined by ``n_candidates``.

Read more in the :ref:`User guide<successive_halving_user_guide>`.

.. note::

  This estimator is still **experimental** for now: the predictions
  and the API might change without any deprecation cycle. To use it,
  you need to explicitly import